In [1]:
import pandas as pd
import re
from tqdm import tqdm

tqdm.pandas()

## Задаем гиперпараметры

In [2]:
N = 50 # numver of samples to compare
SIMILAR_INDEX = 0.3 # using for detecting similar news 

### Готовим данные

In [3]:
data = pd.read_csv('news.csv.gzip', sep=';', compression='gzip')
data.head()

,Unnamed: 0,title,content,url,source
0,0,Майкл Джордан признан лучшим игроком в истории...,"Двукратный олимпийский чемпион, американец Май...",http://www.vesti.ru/doc.html?id=3264755,VestiNewsParser
1,1,ОАК сократила сроки импортозамещения компонент...,"""Объединенная авиастроительная корпорация"" (ОА...",http://www.vesti.ru/doc.html?id=3264736,VestiNewsParser
2,2,Китай выступил против продажи Францией оружия ...,"Китай предупредил Францию, чтобы та не продава...",http://www.vesti.ru/doc.html?id=3264743,VestiNewsParser
3,3,"В Чувашии на оснащение ""Точек роста"" выделят ...",В Чувашии на базе 42 школ создадут Центры обра...,http://www.vesti.ru/doc.html?id=3264744,VestiNewsParser
4,4,"Следовавший из Петербурга в Уфу самолет ""Росси...",Вылетевший из Санкт-Петербурга в Уфу самолет а...,http://www.vesti.ru/doc.html?id=3264746,VestiNewsParser


In [3]:
number_rows_from_other_sources = 300

d = pd.DataFrame()
for source in data['source'].unique():
    d = pd.concat([d, data[data['source'] == source][:number_rows_from_other_sources]], ignore_index=True)

data = d
data.shape

(900, 4)

In [4]:
def get_shinglers(text, N):
    shinglers_list = list()

    text = re.sub(r"\s{1,}", '_', text)  # Replace 1 or more spaces on _
    
    text_length = len(text)
    for index in range(text_length):
        shingler_end = index + N
        if shingler_end > text_length:
            shingler_end = text_length - index
        shingler = text[index:index+N]
        shinglers_list.append(shingler)
    
    return shinglers_list


print(get_shinglers('В Чувашии   на оснащение    "Точек роста" в', 10)) # testing

['В_Чувашии_', '_Чувашии_н', 'Чувашии_на', 'увашии_на_', 'вашии_на_о', 'ашии_на_ос', 'шии_на_осн', 'ии_на_осна', 'и_на_оснащ', '_на_оснаще', 'на_оснащен', 'а_оснащени', '_оснащение', 'оснащение_', 'снащение_"', 'нащение_"Т', 'ащение_"То', 'щение_"Точ', 'ение_"Точе', 'ние_"Точек', 'ие_"Точек_', 'е_"Точек_р', '_"Точек_ро', '"Точек_рос', 'Точек_рост', 'очек_роста', 'чек_роста"', 'ек_роста"_', 'к_роста"_в', '_роста"_в', 'роста"_в', 'оста"_в', 'ста"_в', 'та"_в', 'а"_в', '"_в', '_в', 'в']


In [5]:
data['shinglers'] = data['title'].progress_apply(get_shinglers, args=(5,))
data.head()

100%|██████████| 15000/15000 [00:00<00:00, 31625.42it/s]


,Unnamed: 0,title,content,url,source,shinglers
0,0,Майкл Джордан признан лучшим игроком в истории...,"Двукратный олимпийский чемпион, американец Май...",http://www.vesti.ru/doc.html?id=3264755,VestiNewsParser,"[Майкл, айкл_, йкл_Д, кл_Дж, л_Джо, _Джор, Джо..."
1,1,ОАК сократила сроки импортозамещения компонент...,"""Объединенная авиастроительная корпорация"" (ОА...",http://www.vesti.ru/doc.html?id=3264736,VestiNewsParser,"[ОАК_с, АК_со, К_сок, _сокр, сокра, ократ, кра..."
2,2,Китай выступил против продажи Францией оружия ...,"Китай предупредил Францию, чтобы та не продава...",http://www.vesti.ru/doc.html?id=3264743,VestiNewsParser,"[Китай, итай_, тай_в, ай_вы, й_выс, _выст, выс..."
3,3,"В Чувашии на оснащение ""Точек роста"" выделят ...",В Чувашии на базе 42 школ создадут Центры обра...,http://www.vesti.ru/doc.html?id=3264744,VestiNewsParser,"[В_Чув, _Чува, Чуваш, уваши, вашии, ашии_, шии..."
4,4,"Следовавший из Петербурга в Уфу самолет ""Росси...",Вылетевший из Санкт-Петербурга в Уфу самолет а...,http://www.vesti.ru/doc.html?id=3264746,VestiNewsParser,"[Следо, ледов, едова, довав, овавш, вавши, авш..."


In [6]:
all_shinglers = list()

for sh in data['shinglers']:
    all_shinglers.extend(sh)

print(f"Всего шинглеров: {len(all_shinglers)}")
all_shinglers = set(all_shinglers)
print(f"Уникальных: {len(all_shinglers)}")

Всего шинглеров: 1101178
Уникальных: 123308


## Basic Minshingle

In [7]:
## This method work only if you have >25GB RAM

#new_rows = [{key: 1 for key in row} for row in tqdm(list(data['shinglers']))]
#shinglers_matrix = pd.DataFrame(new_rows, columns=all_shinglers)

100%|██████████| 15000/15000 [00:00<00:00, 136328.01it/s]


In [ ]:
shinglers_matrix = pd.DataFrame(columns=all_shinglers)

for _, row in data.iterrows():
    new_row = {key: 1 for key in row['shinglers']}
    new_row['title'] = row['title']
    shinglers_matrix = shinglers_matrix.append([new_row], ignore_index=True)

In [8]:
#shinglers_matrix = shinglers_matrix.set_index('title')
shinglers_matrix = shinglers_matrix.fillna(0)
shinglers_matrix.head()

,в_вре,яет_т,"ют,_к",о_смя,иптиз,то_да,ентке,нитар,ю_кры,рня_в,...,ика_У,женер,-порн,жайши,ер_Кэ,уанов,знак_,ел_и_,дкой:,я_обя
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
shinglers_matrix.to_csv('shinglers_matrix.csv.gzip', sep=';', compression='gzip')

### Реализация

In [10]:
result = {} # row_index: signature_list with len = n
printable_result = {}


for _ in tqdm(range(N)):  # Do N signature samples
    transpose = shinglers_matrix.T
    random_sorted = transpose.sample(frac=1)

    for docs_columns in random_sorted:
        signature = list(random_sorted[docs_columns]).index(1)  # get index 1st shingler
        if docs_columns not in result:
            result[docs_columns] = list()
        result[docs_columns].append(signature)

data['minshingle'] = pd.Series(result)
data.head()

100%|██████████| 50/50 [2:34:36<00:00, 185.53s/it]  


,Unnamed: 0,title,content,url,source,shinglers,minshingle
0,0,Майкл Джордан признан лучшим игроком в истории...,"Двукратный олимпийский чемпион, американец Май...",http://www.vesti.ru/doc.html?id=3264755,VestiNewsParser,"[Майкл, айкл_, йкл_Д, кл_Дж, л_Джо, _Джор, Джо...","[442, 1321, 2603, 4912, 2971, 2328, 1270, 672,..."
1,1,ОАК сократила сроки импортозамещения компонент...,"""Объединенная авиастроительная корпорация"" (ОА...",http://www.vesti.ru/doc.html?id=3264736,VestiNewsParser,"[ОАК_с, АК_со, К_сок, _сокр, сокра, ократ, кра...","[1833, 1977, 1643, 916, 378, 284, 7036, 1877, ..."
2,2,Китай выступил против продажи Францией оружия ...,"Китай предупредил Францию, чтобы та не продава...",http://www.vesti.ru/doc.html?id=3264743,VestiNewsParser,"[Китай, итай_, тай_в, ай_вы, й_выс, _выст, выс...","[3743, 3871, 1594, 1763, 1912, 768, 3838, 853,..."
3,3,"В Чувашии на оснащение ""Точек роста"" выделят ...",В Чувашии на базе 42 школ создадут Центры обра...,http://www.vesti.ru/doc.html?id=3264744,VestiNewsParser,"[В_Чув, _Чува, Чуваш, уваши, вашии, ашии_, шии...","[723, 332, 4082, 35, 1816, 2470, 2482, 1517, 1..."
4,4,"Следовавший из Петербурга в Уфу самолет ""Росси...",Вылетевший из Санкт-Петербурга в Уфу самолет а...,http://www.vesti.ru/doc.html?id=3264746,VestiNewsParser,"[Следо, ледов, едова, довав, овавш, вавши, авш...","[3, 2929, 2236, 1862, 1851, 3924, 2182, 1312, ..."


In [11]:
def count_sim(x, y):
    all_length = len(x) + len(y)
    intersection = set(x).intersection(set(y))
    return len(intersection) / all_length

data['similarity'] = [[] for _ in range(data.shape[0])] # save empty arrays woth similar news

for index_1, row_1 in tqdm(data.iterrows(), total=data.shape[0]):
    minshingle_1 = row_1['minshingle']
    row_1['similarity'] = []
    for index_2, row_2 in data.iterrows():
        minshingle_2 = row_2['minshingle']
        if index_1 != index_2:
            if count_sim(minshingle_1, minshingle_2) >= SIMILAR_INDEX:
                row_1['similarity'].append(index_2)

  100%|██████████| 15000/15000 [8:47:09<00:00,  2.24s/it]

### Похожая новость:

In [14]:
similar_index = None
for index, value in data['similarity'].items():
    if len(value) > 0 and 'meduza' not in source:
        similar_index = index
        break

example_row = data.iloc[[index]]
example_row

,title,content,url,source,shinglers,minshingle,similarity
37,Данные клиентов курьерской службы слили в сеть,В интернете на продажу выставили базу клиентов...,http://www.vesti.ru/doc.html?id=3264708,VestiNewsParser,"[Данны, анные, нные_, ные_к, ые_кл, е_кли, _кл...","[1147, 62, 167, 185, 276, 80, 300, 458, 470, 8...",[412]


In [15]:
similar_index = None
for index, value in data['similarity'].items():
    if len(value) > 0:
        similar_index = index
        break

example_row = data.iloc[[index]]

print('Исходная новость:\n')
print(str(example_row['content'].item()))

Исходная новость:

В интернете на продажу выставили базу клиентов курьерской службы СДЭК, сообщает "КоммерсантЪ" со ссылкой на телеграм-канал In4security. За 70 тысяч руб. хакеры предлагают купить данные 9 миллионов клиентов. Среди слитой информации — данные о доставке и местонахождении грузов и сведения о покупателях, включая ИНН. База актуальная: на скриншотах, которые продавец прислал автору "Телеграм-канала", значится дата 8 мая, однако никаких подтверждений в подлинности базы пока нет.  В самой компании СДЭК утечку данных клиентов отрицают. Представитель курьерской службы в комментарии изданию отметил, что утечка могла произойти на сторонних ресурсах, которые собирают данные, в том числе на государственных агрегаторах. Представители фирмы отметили, что копирование базы данных настолько большого размера, как у СДЭК, не прошло бы незамеченным: при подобном скачке сотрудники безопасности сразу видят, кто и откуда скачивает данные. Если же база подлинная, то есть вероятность увеличени

In [16]:
similar_rows = list(example_row['similarity'])[0][:5]

similar_news = list()
for s in similar_rows:
    row = data.iloc[[s]]
    similar_news.append(row['content'].item())

print('Похожие новости:\n')
for n in similar_news:
    print(f"{n}\n")

Похожие новости:

Персональные данные более чем девяти миллионов клиентов курьерской службы доставки СДЭК оказались выставлены на продажу. Как сообщает Telegram-канал in4security, стоимость онлайн-доступа к базе данных составляет 70 тысяч рублей.«Интерфакс» отмечает, что в базе есть информация о доставке и местонахождении грузов, а также личные сведения клиентов курьерской службы, включая их ИНН.При этом в СДЭК отрицают утечку. Представитель компании заявил, что личные сведения собирает множество фирм, в их числе — государственные агрегаторы, поэтому утечка могла произойти с какого-либо другого ресурса. Кроме того, он обратил внимание на то, что в сети существует множество мошеннических сайтов, действующих от имени СДЭК.Ранее в мае сообщалось, что в сеть утекли данные миллионов пользователей вебкам-сайта Cam4. По словам группы исследователей Safety Detectives, они смогли получить доступ к почтовым адресам, именам и фамилиям, данным кредитных карт пользователей, а также их переписке в с